In [56]:
import gurobipy as gp
from gurobipy import GRB

# Define problem parameters
I = 20  # Number of elective surgeries
B = 10  # Number of available OR blocks
L =300   # Duration of each OR block (in minutes)

# Surgery durations and costs
d = [140, 60, 150, 60, 120, 50, 170, 90, 100, 140, 100, 70, 150, 140, 160, 100, 60, 160, 80, 80]# duration of surgeries
c = [300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300]# Cost of surgeries
c_p = [500,500,500,500,500,500,500,500,500,500,500,500,500,500,500,500,500,500,500,500,500]# Cost of postponing
c_o = 5  # Cost of overtime per minute

# Create the Gurobi model
model = gp.Model("Elective Surgery Scheduling")

# Define decision variables
y = model.addVars(I, B, range(B), vtype=GRB.BINARY, name="y")
t = model.addVars(B, range(B), vtype=GRB.CONTINUOUS, name="t")
o = model.addVars(B, vtype=GRB.CONTINUOUS, name="o")

# Objective function
obj1 = gp.quicksum(c[i] * y[i, b, p] for i in range(I) for b in range(B) for p in range(B))
obj2 = gp.quicksum(c_p[i] * (1 - gp.quicksum(y[i, b, p] for b in range(B) for p in range(B))) for i in range(I))
obj3 = c_o * gp.quicksum(o[b] for b in range(B))
obj = obj1 + obj2 + obj3
model.setObjective(obj, GRB.MINIMIZE)

# Constraints
model.addConstrs(gp.quicksum(y[i, b, p] for b in range(B) for p in range(B)) == 1 for i in range(I))
model.addConstrs(gp.quicksum(y[i, b, p] for i in range(I)) <= 1 for b in range(B) for p in range(B))
model.addConstrs(t[b, p] == gp.quicksum(t[b, p-1] + d[i] * y[i, b, p-1] for i in range(I)) for b in range(B) for p in range(1, B))
model.addConstrs(t[b, p] >= 0 for b in range(B))
model.addConstrs(o[b] == gp.quicksum(t[b, p] + d[i] * y[i, b, p] for i in range(I) for p in range(B)) - L for b in range(B))
model.addConstrs(o[b] >= 0 for b in range(B))
model.addConstrs(t[b,p]<= L for b in range(B) )


# Solve the problem
model.optimize()

# Print the solution
for i in range(I):
    for b in range(B):
        for p in range(B):
            if y[i, b, p].X > 0.5:
                print(f"Surgery {i} is assigned to block {b},  at time {t[b, p].X}")

# Print the final objective function value (total cost)
print(f"Total cost: {model.ObjVal}")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-1240P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 250 rows, 2110 columns and 8120 nonzeros
Model fingerprint: 0x346a476e
Variable types: 110 continuous, 2000 integer (2000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [5e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Presolve removed 190 rows and 1680 columns
Presolve time: 0.01s
Presolved: 60 rows, 430 columns, 1250 nonzeros
Variable types: 30 continuous, 400 integer (400 binary)
Found heuristic solution: objective 96900.000000
Found heuristic solution: objective 88900.000000

Root relaxation: objective 7.690000e+04, 284 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  